In [1]:
from importlib import reload

import matplotlib.pyplot as plt

import torch
from PIL import Image
import torchvision.transforms as T

import multicrop
import constants
import data
import tokenizer
from datasets import emoreact, enterface
reload(multicrop)
reload(data)
reload(constants)
reload(tokenizer)
reload(emoreact)
reload(enterface)
from data import load_data 
from tokenizer import save_video_frames_tensors
from constants import *



BATCH_SZ = 3
SPLIT = [1, 0.0, 0.0]
dataset_to_use = enterface




In [2]:

train_dl, val_dl, test_dl, ss  = load_data(dataset_to_use, 
                                       batch_sz=BATCH_SZ,
                                       train_val_test_split=SPLIT,
                                       force_audio_aspect=True
                                       )
def get_image_seq(crops, i, rgb=True):

    if rgb:
        image_seq = [ 
            crops[i][j].reshape(FRAMES, CHANS, HEIGHT, WIDTH)[k]
                for j in range(BATCH_SZ) for k in range(3)
        ]

    else:
        # image_seq = [
        #     crops[i][j].reshape(CHANS, NUM_MELS, dataset_to_use.MAX_SPEC_SEQ_LEN)
        #         for j in range(BATCH_SZ)
        # ]
        image_seq = [
            crops[i][j].reshape(CHANS, HEIGHT, WIDTH)
                for j in range(BATCH_SZ)
        ]

    return image_seq

def get_stacked_rgb(crops):
    global_crop_seq = []
    for i in range(len(crops)):
        image_seq_rgb = get_image_seq(crops, i, rgb=True)
        # stacked_frames = torch.cat(image_seq_rgb, dim=0)
        # global_crop_seq.append(stacked_frames)
        global_crop_seq  += image_seq_rgb
    global_stacked = torch.cat(global_crop_seq, dim=0)
    global_stacked = global_stacked.reshape(-1, BATCH_SZ, HEIGHT, WIDTH)
    return global_stacked
    
def get_stacked_spec(crops):
    global_crop_seq = []
    for i in range(len(crops)):
        image_seq_rgb = get_image_seq(crops, i, rgb=False)
        # stacked_frames = torch.cat(image_seq_rgb, dim=0)
        # global_crop_seq.append(stacked_frames)
        global_crop_seq  += image_seq_rgb
    global_stacked = torch.cat(global_crop_seq, dim=0)
    # global_stacked = global_stacked.reshape(-1, BATCH_SZ, NUM_MELS, dataset_to_use.MAX_SPEC_SEQ_LEN)
    global_stacked = global_stacked.reshape(-1, BATCH_SZ, HEIGHT, WIDTH)
    return global_stacked

for data in train_dl:
    crops = data["student_spec"]
    crops = data["teacher_rgb"]
    crops = data["student_rgb"]
    crops = data["teacher_spec"]
    # global_stacked = get_stacked_rgb(crops)
    global_stacked = get_stacked_spec(crops)
    break

    # for video, audio in zip(data["teacher_rgb"], data["teacher_spec"]):
    #     video_frames = video[0].reshape(FRAMES, CHANS, HEIGHT, WIDTH)
    #     audio_frame = audio[0]
    #     break
    # break
shape = global_stacked.shape
save_video_frames_tensors(global_stacked, "frames_a", rows=int(shape[0] / NUM_LOCAL_VIEWS))
# save_video_frames_tensors(global_stacked, "frames_v", rows=int(shape[0] / NUM_LOCAL_VIEWS))

# save_video_frames_tensors(audio_frame, "frame_a")
# save_video_frames_tensors(video_frames, "frames_v")
    # total_items = len(global_crops) + len(local_crops)
# stacked_local = torch.cat(local_crops, dim=2)
# plt.imshow(global_stacked.swapaxes(0, 2).swapaxes(0, 1))
# plt.imshow(global_stacked)
# plt.imshow(stacked_local.swapaxes(0, 2).swapaxes(0, 1))

/root/depmbt/data.py:86: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.dataset.iloc[item][0],
/root/depmbt/data.py:87: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.dataset.iloc[item][1],
/root/depmbt/data.py:88: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.dataset.iloc[item][2],
/root/depmbt/data.py:89: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will

torch.Size([3, 3, 224, 224])


/root/miniconda/envs/depmbt/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
